In [1]:
import ultralytics
from ultralytics import YOLO
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [3]:
# Load the CSV file
data = pd.read_csv(r"C:\Users\kbtha\Desktop\Mask Detection\Dataset\train.csv")
print(len(data))
data.head()

15412


,name,x1,x2,y1,y2,classname
0,2756.png,69,126,294,392,face_with_mask
1,2756.png,505,10,723,283,face_with_mask
2,2756.png,75,252,264,390,mask_colorful
3,2756.png,521,136,711,277,mask_colorful
4,6098.jpg,360,85,728,653,face_no_mask


In [5]:
data['classname'].value_counts()

classname
face_with_mask              4180
mask_surgical               2430
mask_colorful               1876
face_no_mask                1569
face_other_covering         1372
eyeglasses                   914
hat                          823
sunglasses                   358
hair_net                     287
scarf_bandana                260
goggles                      192
helmet                       187
hijab_niqab                  173
face_shield                  160
hood                         159
face_with_mask_incorrect     150
balaclava_ski_mask           134
turban                        94
gas_mask                      55
other                         39
Name: count, dtype: int64

In [7]:
filter_data = data[data['classname'].isin(['face_with_mask', 'face_no_mask'])]
filter_data = filter_data.reset_index(drop=True)
print(len(filter_data))
filter_data.head()

5749


,name,x1,x2,y1,y2,classname
0,2756.png,69,126,294,392,face_with_mask
1,2756.png,505,10,723,283,face_with_mask
2,6098.jpg,360,85,728,653,face_no_mask
3,4591.png,239,9,522,342,face_with_mask
4,5525.jpg,262,18,484,319,face_no_mask


In [9]:
filter_data = filter_data.drop_duplicates()
filter_data = filter_data.reset_index(drop=True)
print(len(filter_data))
filter_data.head()

5748


,name,x1,x2,y1,y2,classname
0,2756.png,69,126,294,392,face_with_mask
1,2756.png,505,10,723,283,face_with_mask
2,6098.jpg,360,85,728,653,face_no_mask
3,4591.png,239,9,522,342,face_with_mask
4,5525.jpg,262,18,484,319,face_no_mask


In [11]:
# Preprocess images and labels
image_folder = r"C:\Users\kbtha\Desktop\Mask Detection\Dataset\Medical mask\images"
output_img_folder = "C:\\Users\\kbtha\\Desktop\\Mask Detection\\images"
output_label_folder = r"C:\Users\kbtha\Desktop\Mask Detection\labels"
os.makedirs(output_img_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)

for index, row in filter_data.iterrows():
    img_path = os.path.join(image_folder, row['name'])
    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        continue
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to load image: {img_path}")
        continue
        
    # Save the image to the output folder
    output_path = os.path.join(output_img_folder, row['name'])
    cv2.imwrite(output_path, img)
    
    image_height, image_width = img.shape[:2]
    x1 = row['x1'] 
    y1 = row['x2']
    x2 = row['y1'] 
    y2 = row['y2']
    x_center = (x1 + x2) / 2 / image_width
    y_center = (y1 + y2) / 2 / image_height
    width = (x2 - x1) / image_width
    height = (y2 - y1) / image_height

    # Generate YOLO annotation
    yolo_annotation = f"{1 if row['classname'] == 'face_with_mask' else 0} {x_center} {y_center} {width} {height}"

    # Write the annotation to the .txt file
    output_path1 = os.path.join(output_label_folder, f"{row['name']}.txt")
    with open(output_path1, 'a') as f:
        f.write(yolo_annotation + '\n')

print("Image and label processing completed.")

Image and label processing completed.


In [13]:
output_label_folder = r"C:\Users\kbtha\Desktop\Mask Detection\labels"
# Rename files by stripping out the image extension
for filename in os.listdir(output_label_folder):
    if filename.endswith(".txt"):
        new_filename = f"{os.path.splitext(os.path.splitext(filename)[0])[0]}.txt"
        old_path = os.path.join(output_label_folder, filename)
        new_path = os.path.join(output_label_folder, new_filename)
        
        # Only rename if the new file does not exist
        if not os.path.exists(new_path):
            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} -> {new_path}")
        else:
            print(f"Skipped (file exists): {new_filename}")

Renamed: C:\Users\kbtha\Desktop\Mask Detection\labels\1801.jpg.txt -> C:\Users\kbtha\Desktop\Mask Detection\labels\1801.txt
Renamed: C:\Users\kbtha\Desktop\Mask Detection\labels\1802.jpg.txt -> C:\Users\kbtha\Desktop\Mask Detection\labels\1802.txt
Renamed: C:\Users\kbtha\Desktop\Mask Detection\labels\1803.jpg.txt -> C:\Users\kbtha\Desktop\Mask Detection\labels\1803.txt
Renamed: C:\Users\kbtha\Desktop\Mask Detection\labels\1804.jpg.txt -> C:\Users\kbtha\Desktop\Mask Detection\labels\1804.txt
Renamed: C:\Users\kbtha\Desktop\Mask Detection\labels\1805.jpg.txt -> C:\Users\kbtha\Desktop\Mask Detection\labels\1805.txt
Renamed: C:\Users\kbtha\Desktop\Mask Detection\labels\1806.jpg.txt -> C:\Users\kbtha\Desktop\Mask Detection\labels\1806.txt
Renamed: C:\Users\kbtha\Desktop\Mask Detection\labels\1808.jpg.txt -> C:\Users\kbtha\Desktop\Mask Detection\labels\1808.txt
Renamed: C:\Users\kbtha\Desktop\Mask Detection\labels\1809.jpg.txt -> C:\Users\kbtha\Desktop\Mask Detection\labels\1809.txt
Renamed: